In [1]:
import tweepy
import csv
import pandas as pd
import re

In [2]:
consumer_key = ' '
consumer_secret = ' '
access_token = ' '
access_token_secret = ' '

In [11]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

csvFile = open('hilton_tweets.csv', 'w')

csvWriter = csv.writer(csvFile)

tweets = []
csvWriter.writerow(["timestamp","tweet"])
for tweet in tweepy.Cursor(api.search,q="#hilton",lang="en").items():
    tweets.append((tweet.created_at, tweet.text))
    csvWriter.writerow([tweet.created_at, tweet.text.strip().encode('utf-8')])

In [12]:
print(tweets)

[(datetime.datetime(2018, 12, 7, 21, 35, 34), '❄️Christmas is around the corner and we had today the first Christmas Party with the care committee team! Some food… https://t.co/wLGze3jGk3'), (datetime.datetime(2018, 12, 7, 21, 25, 12), '🎵Deck the Halls🎵\n.\n.\n.\n#HamptonDT #Downtownhotel #decorations #lights #icicle #bear #snowman #Hilton #wreath #hotel… https://t.co/J6XWT8gYXb'), (datetime.datetime(2018, 12, 7, 21, 24, 7), 'RT @RoyalDornochPro: There are some beautiful sights to be seen @RoyalDornochGC Bunker reconstruction being one of them #notthesamebunker #…'), (datetime.datetime(2018, 12, 7, 21, 18, 55), 'Hilton Worldwide Is Set To Become A Force In the Luxury Hotel Market via @forbes https://t.co/94oikhfCA9… https://t.co/7Lp9ymeqUl'), (datetime.datetime(2018, 12, 7, 21, 12, 56), 'RT @RoyalDornochPro: There are some beautiful sights to be seen @RoyalDornochGC Bunker reconstruction being one of them #notthesamebunker #…'), (datetime.datetime(2018, 12, 7, 21, 11, 55), 'There are s

In [13]:
sentiment_scores = {}

with open ("tidytext_sentiments.txt",'r') as input_file :
    next(input_file)
    for line in input_file.readlines():     
        word, sentiment, lexicon = line.split("\t")
        if sentiment == "positive" :
            sentiment_scores[word] = 1
        elif sentiment == "negative" :
            sentiment_scores[word] = -1 

assert(len(sentiment_scores) > 0)

In [14]:
running_score = 0
scores = []
for idx, (date, content) in enumerate(tweets) :
    for word in content.split():
        if word.lower() in sentiment_scores :
            running_score += sentiment_scores[word.lower()]
    
    scores.append((date, content, running_score))
print(scores)

[(datetime.datetime(2018, 12, 7, 21, 35, 34), '❄️Christmas is around the corner and we had today the first Christmas Party with the care committee team! Some food… https://t.co/wLGze3jGk3', 0), (datetime.datetime(2018, 12, 7, 21, 25, 12), '🎵Deck the Halls🎵\n.\n.\n.\n#HamptonDT #Downtownhotel #decorations #lights #icicle #bear #snowman #Hilton #wreath #hotel… https://t.co/J6XWT8gYXb', 0), (datetime.datetime(2018, 12, 7, 21, 24, 7), 'RT @RoyalDornochPro: There are some beautiful sights to be seen @RoyalDornochGC Bunker reconstruction being one of them #notthesamebunker #…', 2), (datetime.datetime(2018, 12, 7, 21, 18, 55), 'Hilton Worldwide Is Set To Become A Force In the Luxury Hotel Market via @forbes https://t.co/94oikhfCA9… https://t.co/7Lp9ymeqUl', 2), (datetime.datetime(2018, 12, 7, 21, 12, 56), 'RT @RoyalDornochPro: There are some beautiful sights to be seen @RoyalDornochGC Bunker reconstruction being one of them #notthesamebunker #…', 4), (datetime.datetime(2018, 12, 7, 21, 11, 55

In [15]:
df = pd.DataFrame(scores,columns=['Timestamp','Tweet','Sentiment'])
print(df)

              Timestamp                                              Tweet  \
0   2018-12-07 21:35:34  ❄️Christmas is around the corner and we had to...   
1   2018-12-07 21:25:12  🎵Deck the Halls🎵\n.\n.\n.\n#HamptonDT #Downtow...   
2   2018-12-07 21:24:07  RT @RoyalDornochPro: There are some beautiful ...   
3   2018-12-07 21:18:55  Hilton Worldwide Is Set To Become A Force In t...   
4   2018-12-07 21:12:56  RT @RoyalDornochPro: There are some beautiful ...   
5   2018-12-07 21:11:55  There are some beautiful sights to be seen @Ro...   
6   2018-12-07 21:01:13  @neal_katyal The worst .@POTUS of all time. It...   
7   2018-12-07 20:08:21  Hilton Honors 101: How to Make the Most of Thi...   
8   2018-12-07 19:45:50  Do you want to save money??\nhttps://t.co/zJEp...   
9   2018-12-07 18:43:14  RT @ghadasoliman1: Fun with our Animation Team...   
10  2018-12-07 18:13:45  Fun with our Animation Team at Hilton Sharks B...   
11  2018-12-07 18:13:11  Julbord for a private group #julbord #4

In [16]:
df["Tweet"] = df["Tweet"].str.replace(r'RT @'," ",regex=True)
df["Tweet"] = df["Tweet"].str.replace(r'#'," ",regex=True)
df["Tweet"] = df["Tweet"].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
print(df["Tweet"])

0      ❄️Christmas is around the corner and we had to...
1      🎵Deck the Halls🎵\n.\n.\n.\n HamptonDT  Downtow...
2       RoyalDornochPro: There are some beautiful sig...
3      Hilton Worldwide Is Set To Become A Force In t...
4       RoyalDornochPro: There are some beautiful sig...
5      There are some beautiful sights to be seen @Ro...
6      @neal_katyal The worst .@POTUS of all time. It...
7      Hilton Honors 101: How to Make the Most of Thi...
8                          Do you want to save money??\n
9       ghadasoliman1: Fun with our Animation Team at...
10     Fun with our Animation Team at Hilton Sharks B...
11     Julbord for a private group  julbord  430perso...
12     Fun with our Animation Team at Hilton Sharks B...
13      DTDenver_SN: Look at Chef Chip supporting us ...
14     I recently highlighted @Hilton_Addis on the @H...
15      PragueConBureau:  Hilton Prague Old Town unde...
16     An amazing night and a few photos from the Chr...
17     Buy Tickets for our amaz

In [17]:
df["Tweet"] = df["Tweet"].str.lower()
print(df["Tweet"])

0      ❄️christmas is around the corner and we had to...
1      🎵deck the halls🎵\n.\n.\n.\n hamptondt  downtow...
2       royaldornochpro: there are some beautiful sig...
3      hilton worldwide is set to become a force in t...
4       royaldornochpro: there are some beautiful sig...
5      there are some beautiful sights to be seen @ro...
6      @neal_katyal the worst .@potus of all time. it...
7      hilton honors 101: how to make the most of thi...
8                          do you want to save money??\n
9       ghadasoliman1: fun with our animation team at...
10     fun with our animation team at hilton sharks b...
11     julbord for a private group  julbord  430perso...
12     fun with our animation team at hilton sharks b...
13      dtdenver_sn: look at chef chip supporting us ...
14     i recently highlighted @hilton_addis on the @h...
15      pragueconbureau:  hilton prague old town unde...
16     an amazing night and a few photos from the chr...
17     buy tickets for our amaz

In [18]:
df.to_csv("hilton_clean_tweets.csv", sep=',', encoding='utf-8')